In [1]:
import sys

sys.path.append('../../code/')
import os
import json
from datetime import datetime
import time
from math import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

import igraph as ig

import random as random

from collections import *

from load_data import load_citation_network_igraph, case_info

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, log_loss
from sklearn import preprocessing
from sklearn import neighbors


%load_ext autoreload
%autoreload 2
%matplotlib inline

data_dir = '../../data/'
court_name = 'scotus'

# %load ../standard_import.txt
from __future__ import division
import matplotlib as mpl

pd.set_option('display.notebook_repr_html', False)

%matplotlib inline
plt.style.use('seaborn-white')

In [2]:
G = load_citation_network_igraph(data_dir, court_name)
print 'loaded %s network with %d cases and %d edges' % (court_name, len(G.vs), len(G.es))

0 seconds for 250465 edges
loaded scotus network with 33248 cases and 250465 edges


# SCOTUS Subgraphs by decade

In [3]:
# start from 1800

sub_G_dict = {}
scotus_years = range(1800, 2021) # scotus years actually from 1754-2016
scotus_decades = [year for year in scotus_years if year % 10 == 0] # 1800, ... , 2000, 2010, 2020

dict_decades_subgraphs = OrderedDict() # key: decade, value: subgraph with vertices less than that decade year
for i in scotus_decades:
    sub_vs = G.vs.select(year_lt=i)
    sub_G = G.subgraph(sub_vs) # IMPORTANT NOTE: THESE NEW SUBGRAPHS RE-INDEX (DIFFERENT INDICES THAN G_2)
    dict_decades_subgraphs[i] = sub_G
    print i, ": ", len(sub_G.vs), "vertices and ", len(sub_G.es), " edges"

1800 :  252 vertices and  17  edges
1810 :  478 vertices and  48  edges
1820 :  803 vertices and  120  edges
1830 :  1148 vertices and  156  edges
1840 :  1584 vertices and  294  edges
1850 :  1956 vertices and  474  edges
1860 :  2680 vertices and  831  edges
1870 :  3475 vertices and  884  edges
1880 :  5319 vertices and  2271  edges
1890 :  7902 vertices and  10255  edges
1900 :  10446 vertices and  25674  edges
1910 :  12463 vertices and  37864  edges
1920 :  14880 vertices and  52274  edges
1930 :  16887 vertices and  67361  edges
1940 :  18585 vertices and  86576  edges
1950 :  20079 vertices and  106644  edges
1960 :  21329 vertices and  118369  edges
1970 :  23642 vertices and  136684  edges
1980 :  25734 vertices and  166573  edges
1990 :  27848 vertices and  199818  edges
2000 :  29206 vertices and  221713  edges
2010 :  32505 vertices and  238179  edges
2020 :  33248 vertices and  250465  edges


# Create list of tuples of every present edge (1) with info (indices, years)

In [4]:
time1 = time.time()
edge_tuple_dict = {}
#for each subgraph makes a list of present edges (1)
for key in dict_decades_subgraphs:
    decade_sub_G = dict_decades_subgraphs[key]
    edge_tuple = decade_sub_G.get_edgelist() #returns a list of edges of the form (citing_index, cited_index)
    for i in range(0,len(edge_tuple)):
        #adds two items to each tuple which corresponds to the citing year and cited year
        #each list item is now in the form (1, citing_index, cited_index, citing_year, cited_year)
        edge_tuple[i] = (1, decade_sub_G.vs(edge_tuple[i][0])['name'][0]) + edge_tuple[i] + (decade_sub_G.vs(edge_tuple[i][0])['year'][0], decade_sub_G.vs(edge_tuple[i][1])['year'][0])
    edge_tuple_dict[key] = edge_tuple
time2 = time.time()
print "this took " + str(time2-time1) + " seconds"

this took 397.924999952 seconds


# Tidy Data Frame (N1)

In [5]:
time1 = time.time()
df_dict = {}
list_of_column_names = ["edge", "citing_name" , "citing_index", "cited_index", "citing_year", "cited_year"]
#for each list of edges converts it into a df
for key in edge_tuple_dict:
    edge_tuples = edge_tuple_dict[key]
    df = pd.DataFrame(edge_tuples, columns=list_of_column_names)
    df_dict[key] = df 
time2 = time.time()
print "this took " + str(time2-time1) + " seconds"

this took 1.73600006104 seconds


In [ ]:
#save tidy df
#df.to_csv('1925_scotus_logreg_table.csv')

In [6]:
print len(G.get_edgelist())
print len(set(G.get_edgelist()))

250465
250465


# Create list of tuples of not present edges (0) with info (indices, years)

# old code, where we had set(vertices) in the while loop -- 0.45 seconds

In [70]:
time1 = time.time()
edge_tuple_dict_0 = {}

# for each subgraph make a list of non-present edges
decade_sub_G = dict_decades_subgraphs[1850]

vertices = decade_sub_G.vs

one_edges = set(decade_sub_G.get_edgelist())
zero_edges = set([])
time3 = time.time()

while len(zero_edges) < len(one_edges):
    new_edge_list = random.sample(set(vertices), 2)
    
    new_edge = (new_edge_list[0],new_edge_list[1])
    final_edge = (0, decade_sub_G.vs(new_edge[0].index)['name'][0], new_edge[0].index, new_edge[1].index, decade_sub_G.vs(new_edge[0].index)['year'][0], decade_sub_G.vs(new_edge[1].index)['year'][0])
    # todo: explain this long if statement (last conditional makes assumption that we will not use edge in same year)
    if new_edge not in one_edges and final_edge not in zero_edges and final_edge[4]>final_edge[5]:
        zero_edges.add(final_edge)
time4 = time.time()
print 1850, ":", len(zero_edges), "NON-present edges", "--took", time4-time3, "seconds--"
edge_tuple_dict_0[1850] = zero_edges

time2 = time.time()
print "this took " + str(time2-time1) + " seconds"

1850 : 474 NON-present edges --took 0.458000183105 seconds--
this took 0.460000038147 seconds


# np.random.choice(vertices)--5.8 seconds vs. 
# random.sample(vertices_set)--0.06 seconds

In [93]:
time1 = time.time()
edge_tuple_dict_0 = {}

# for each subgraph make a list of non-present edges
decade_sub_G = dict_decades_subgraphs[1850]

vertices = decade_sub_G.vs
vertices_set = set(vertices)

one_edges = set(decade_sub_G.get_edgelist())
zero_edges = set([])
time3 = time.time()

while len(zero_edges) < len(one_edges):
    #temp = np.random.choice(vertices, size=2, replace=False)
    temp = random.sample(vertices_set, 2)
    
    new_edge = (temp[0], temp[1])
    final_edge = (0, decade_sub_G.vs(new_edge[0].index)['name'][0], new_edge[0].index, new_edge[1].index, decade_sub_G.vs(new_edge[0].index)['year'][0], decade_sub_G.vs(new_edge[1].index)['year'][0])
    # todo: explain this long if statement (last conditional makes assumption that we will not use edge in same year)
    if new_edge not in one_edges and final_edge not in zero_edges and final_edge[4]>final_edge[5]:
        zero_edges.add(final_edge)
time4 = time.time()
print 1850, ":", len(zero_edges), "NON-present edges", "--took", time4-time3, "seconds--"
edge_tuple_dict_0[1850] = zero_edges

time2 = time.time()
print "this took " + str(time2-time1) + " seconds"

1850 : 474 NON-present edges --took 5.58799982071 seconds--
this took 5.59100008011 seconds


# np.random.choice(indices)-- 0.28 vs.
# random.sample(indices_set)--0.08

In [92]:
time1 = time.time()
edge_tuple_dict_0 = {}

# for each subgraph make a list of non-present edges
decade_sub_G = dict_decades_subgraphs[1850]

vertices = decade_sub_G.vs
indices = range(0, len(vertices))
indices_set = set(indices)

one_edges = set(decade_sub_G.get_edgelist())
zero_edges = set([])
time3 = time.time()

while len(zero_edges) < len(one_edges):
    temp = np.random.choice(indices, size=2, replace=False)
    #temp = random.sample(indices_set, 2)
    
    new_edge = (temp[0], temp[1])
    final_edge = (0, decade_sub_G.vs(new_edge[0])['name'][0], new_edge[0], new_edge[1], decade_sub_G.vs(new_edge[0])['year'][0], decade_sub_G.vs(new_edge[1])['year'][0])
    # todo: explain this long if statement (last conditional makes assumption that we will not use edge in same year)
    if new_edge not in one_edges and final_edge not in zero_edges and final_edge[4]>final_edge[5]:
        zero_edges.add(final_edge)
time4 = time.time()
print 1850, ":", len(zero_edges), "NON-present edges", "--took", time4-time3, "seconds--"
edge_tuple_dict_0[1850] = zero_edges

time2 = time.time()
print "this took " + str(time2-time1) + " seconds"

1850 : 474 NON-present edges --took 0.27999997139 seconds--
this took 0.281999826431 seconds
